In [6]:
import random
import sys
import os
import cv2


project_root = r'C:\Users\huber_15ivafg\PycharmProjects\Projekt_Inzynierski'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")
print(f"sys.path updated: {project_root in sys.path}")

from abc import ABC, abstractmethod


class NucleiTest(ABC):
    _list_of_points = []
    _counter = 0
    _number_of_points = 0

    def __init__(self,
                 center,
                 axes_size,
                 angle= random.randint(0, 360),
                 color=(160, 83, 179),
                 thickness=-1,
                 border_color=(107, 26, 121),
                 border_thickness=2):

        self.center = center
        self.axes_size = axes_size
        self.angle = angle
        self.color = color
        self.thickness = thickness
        self.border_color = border_color
        self.border_thickness = border_thickness



    def return_center(self):
        return self.center

    def draw_nuclei(self, image):
        cv2.ellipse(image, self.center, self.axes, self.angle, 0, 360, self.color, self.thickness)

        if self.border_thickness > 0:
            cv2.ellipse(image, self.center, self.axes, self.angle, 0, 360, self.border_color, self.border_thickness)

Project root: C:\Users\huber_15ivafg\PycharmProjects\Projekt_Inzynierski
sys.path updated: True


In [7]:
class CancerNucleusTest(NucleiTest):
    def __init__(self, center, axes_size, irregularity=0.3, angle=0, color=(160, 83, 179),
                 thickness=-1, border_color=(107, 26, 121), border_thickness=2):

        super().__init__(
            center=center,
            axes_size=axes_size,
            angle=angle,
            color=color,
            thickness=thickness,
            border_color=border_color,
            border_thickness=border_thickness
        )
        self.irregularity = irregularity

    def draw_nuclei(self, image):
        super().draw_nuclei(image)

In [3]:
import random
from abc import ABC, abstractmethod

from rpds.rpds import List

from src.functions.poisson_sampling import poisson_sampling


class CenterPointsGenerator(ABC):

    def __init__(self, size_x, size_y, number_of_points):
        self.size_x = size_x
        self.size_y = size_y
        self.number_of_points = number_of_points
        self._points = []

        self._counter = 0

        @abstractmethod
        def generate_points(self) -> List[int,int]:
            pass

        def get_next_center(self):
            if not self._points:
                self._points = self.generate_points()

            if self._counter < len(self._points):
                center = self._points[self._counter]
                self._counter += 1
                return center
            else:
                self._counter = 0
                self._points = self.generate_points()
                return self.get_next_center()

class PoissonAlgorithm(CenterPointsGenerator):
    def __init__(self, size_x, size_y, radius=5, k=30):
        super().__init__(size_x,size_y)
        self.radius = radius
        self.k = k

    def generate_points(self):
        return poisson_sampling(self.size_x, self.size_y, self.radius, self.k)

class RandomAlignmentOfCenters(CenterPointsGenerator):
    def __init__(self, size_x, size_y, number_of_points, cell_size):
        super().__init__(size_x, size_y, number_of_points )
        self.cell_size = cell_size

    def generate_points(self):
        points = []
        for _ in range(self.number_of_points):
            x = random.uniform(4 + self.cell_size, self.size_x - self.cell_size - 4)
            y = random.uniform(4 + self.cell_size, self.size_y - self.cell_size - 4)
            p = [int(x), int(y)]
            points.append(p)

        return points

class GaussianAlgorithm(CenterPointsGenerator):
    def __init__(self, size_x, size_y, number_of_points, dev):
        super().__init__(size_x, size_y, number_of_points)
        self.dev = dev

        def generate_points(self):
            points = []
            center_x = self.size_x / 2
            center_y = self.size_y / 2

            for _ in range(self.number_of_points):
                x = random.gauss(center_x, self.size_x * self.dev)
                y = random.gauss(center_y, self.size_y * self.dev)

                x = max(0, min(int(x), self.size_x))
                y = max(0, min(int(y), self.size_y))

                points.append([x, y])

            return points


class ClusteredAlgorithm(CenterPointsGenerator):
    """
    Implements a clustering algorithm based on generated centers.

    This class derives from the `CenterPointsGenerator` base class and provides
    an implementation of an algorithm used for clustering data points into
    groups based on specific criteria. Its purpose is to facilitate
    cluster analysis by leveraging generated centers and applying clustering
    principles to group data effectively. The class provides mechanisms
    to initialize, process, and manage clusters based on the underlying
    algorithm.

    Attributes
    ----------
    max_iterations : int
        The maximum number of iterations allowed for the clustering algorithm
        to converge.
    tolerance : float
        The tolerance level for convergence. Determines the extent of
        allowable changes in cluster centers before stopping iterations.
    clusters : dict
        A dictionary where keys represent cluster identifiers and values
        represent associated data points.
    centers : list
        A list of centers, each representing a cluster's central point in
        the feature space.
    """

    def __init__(self, size_x, size_y, number_of_points, num_clusters = None, dev=40):
        super().__init__(size_x, size_y, number_of_points)
        self.num_clusters = num_clusters
        self.dev = dev

        if num_clusters is None:
            self.num_clusters = random.randint(3, 9)

    def generate_points(self):
        points = []

        cluster_centers = []
        for _ in range(self.num_clusters):
            cx = random.uniform(self.dev, self.size_x - self.dev)
            cy = random.uniform(self.dev, self.size_y - self.dev)
            cluster_centers.append((cx, cy))

        for _ in range(self.number_of_points):
            chosen_center = random.choice(cluster_centers)

            x = random.gauss(chosen_center[0], self.dev)
            y = random.gauss(chosen_center[1], self.dev)

            x = max(0, min(int(x), self.size_x))
            y = max(0, min(int(y), self.size_y))

            points.append([x, y])

        return points

In [8]:
import random
from abc import ABC, abstractmethod

from src.functions.ellipse_params import ellipse_proportion


class Axes(ABC):
    def __init__(self, x, y, size, ratio = 0.65):
        self.x = x
        self.y = y
        self.size = size
        self._ratio = ratio

    @property
    def ratio(self):
        return self._ratio
    @ratio.setter
    def ratio(self, value):
        self._ratio = value

    @abstractmethod
    def generate_axes(self) -> [int, int]:
        pass

    def use_ratio(self):
        return ellipse_proportion(self.size, self.ratio)


class AxesWithNormalDistribution(Axes):
    def __init__(self, x, y, dev):
       super().__init__(x, y)
       self.dev = dev

    def generate_axes(self):
        gen_x = random.gauss(self.x, self.dev)
        gen_y = random.gauss(self.y, self.dev)
        return int(gen_x), int(gen_y)

class AxesWithUniformDistribution(Axes):
    def __init__(self, x, y, dev):
        super().__init__(x, y)
        self.dev = dev

    def generate_axes(self):
        return random.randint(self.x - self.dev,  self.x + self.dev), random.randint(self.y - self.dev, self.y + self.dev)

In [5]:
c1 = CancerNucleusTest()

TypeError: CancerNucleusTest.__init__() missing 2 required positional arguments: 'center' and 'axes_size'